# ARIMA & SARIMA: Final Performance Evaluation and Comparison

This notebook trains and evaluates ARIMA and SARIMA models for forecasting the sum of the next 4 weeks (monthly forecast) of respiratory morbidity rates in Brazilian municipalities. Results are compared with previous deep learning and baseline models.

- **Models:** ARIMA, SARIMA
- **Target:** Sum of next 4 weeks (monthly forecast)
- **Approach:** Same data splits and evaluation as other notebooks for fair comparison.
- **All code is modular and uses the same data pipeline as other models.**

In [11]:
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to sys.path (not the src directory)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")
    import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from src.preprocessing import load_city_data, filter_city, clean_timeseries, prepare_data_for_model
from src.train import evaluate_model, save_predictions, save_metrics
from src.utils import plot_forecast, plot_forecast_error

results_dir = os.path.join('results', 'arima_sarima')
os.makedirs(results_dir, exist_ok=True)
np.random.seed(42)

Project root: c:\Users\pedro\OneDrive - Unesp\Documentos\GitHub\cities-models\cities-models


## Carregamento e Seleção dos Dados

Selecione a cidade para avaliação. O pipeline é idêntico aos outros notebooks.

In [12]:
# Exemplo: Carregar dados de uma cidade (ajuste o caminho conforme necessário)
data_path = '../data/df_base_morb_resp.csv'
df = load_city_data(data_path)
CD_MUN_SELECTED = 3550308  # São Paulo
selected_city_name = "São Paulo" # Define city name for plots
df_city = filter_city(df, cd_mun=CD_MUN_SELECTED)
df_city = clean_timeseries(df_city, target_column='target')
print(f"Selected city shape: {df_city.shape}")

# Ensure 'week' is a PeriodIndex for statsmodels compatibility
if not pd.api.types.is_period_dtype(df_city['week']):
    df_city['week'] = pd.to_datetime(df_city['week'])
    df_city['week'] = df_city['week'].dt.to_period('W')

def make_rolling_targets(series, forecast_horizon):
    return series.rolling(window=forecast_horizon, min_periods=forecast_horizon).sum().shift(-forecast_horizon+1)

model_params = {
    'forecast_horizon': 4,  # Soma das próximas 4 semanas
    'test_size': 40,        # Últimas 40 semanas para teste
}
target_column = 'target'

df_city = df_city.set_index('week')
df_city['target_sum4w'] = make_rolling_targets(df_city[target_column], model_params['forecast_horizon'])
df_city = df_city.dropna(subset=['target_sum4w'])

test_size = model_params['test_size']
train = df_city.iloc[:-test_size]
test = df_city.iloc[-test_size:]
train_series = train['target_sum4w']
test_series = test['target_sum4w']
test_weeks = test.index

Selected city shape: (1200, 11)


## Diagnóstico e Transformações Automáticas (Pré-processamento)

Antes de treinar ARIMA/SARIMA, o notebook verifica e aplica automaticamente as transformações necessárias para garantir estacionariedade e variância estável.

In [13]:
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.stats import boxcox
from scipy.special import inv_boxcox

# Função para testar estacionariedade
def test_stationarity(series, alpha=0.05, verbose=True):
    adf_result = adfuller(series.dropna(), autolag='AIC')
    kpss_result = kpss(series.dropna(), regression='ct', nlags='auto')
    if verbose:
        print(f"ADF Statistic: {adf_result[0]:.4f}, p-value: {adf_result[1]:.4f}")
        print(f"KPSS Statistic: {kpss_result[0]:.4f}, p-value: {kpss_result[1]:.4f}")
    stationary = (adf_result[1] < alpha) and (kpss_result[1] > alpha)
    return stationary, adf_result, kpss_result

# Função para estabilizar variância
def try_boxcox(series):
    shifted = False
    min_val = series.min()
    if min_val <= 0:
        series = series + abs(min_val) + 1
        shifted = True
    y_box, lam = boxcox(series.dropna())
    return y_box, lam, shifted, min_val

# Diagnóstico e transformação
y = train_series.copy()
print("Diagnóstico inicial da série de treino:")
stationary, adf_res, kpss_res = test_stationarity(y)

# 1. Variance stabilization
if y.skew() > 1 or y.kurt() > 5:
    print("Série apresenta alta assimetria ou curtose. Aplicando transformação logarítmica.")
    y_trans = np.log1p(y)
    trans_type = 'log1p'
else:
    try:
        y_box, lam, shifted, min_val = try_boxcox(y)
        if abs(lam - 1) > 0.1:
            print(f"Aplicando transformação Box-Cox (lambda={lam:.2f}).")
            y_trans = pd.Series(y_box, index=y.dropna().index)
            trans_type = f'boxcox:{lam:.2f}'
            if shifted:
                print(f"A série foi deslocada por {abs(min_val)+1} para garantir positividade.")
        else:
            y_trans = y
            trans_type = 'none'
    except Exception as e:
        print(f"Box-Cox falhou: {e}. Usando série original.")
        y_trans = y
        trans_type = 'none'

# 2. Test stationarity after variance stabilization
print(f"\nTestando estacionariedade após transformação ({trans_type}):")
stationary, adf_res, kpss_res = test_stationarity(y_trans)

# 3. Differencing if needed
ndiffs = 0
if not stationary:
    print("Série ainda não estacionária. Aplicando diferenciação de 1ª ordem.")
    y_trans = y_trans.diff().dropna()
    ndiffs += 1
    stationary, adf_res, kpss_res = test_stationarity(y_trans)
    if not stationary:
        print("Ainda não estacionária após 1ª diferença. Aplicando 2ª ordem.")
        y_trans = y_trans.diff().dropna()
        ndiffs += 1
        stationary, adf_res, kpss_res = test_stationarity(y_trans)
    else:
        print("Série estacionária após 1ª diferença.")
else:
    print("Série já estacionária após transformação.")

print(f"\nResumo da transformação: {trans_type}, número de diferenças aplicadas: {ndiffs}")

# Salvar para uso posterior
train_transformed = y_trans
transformation_info = {'type': trans_type, 'ndiffs': ndiffs}

Diagnóstico inicial da série de treino:
ADF Statistic: -4.4071, p-value: 0.0003
KPSS Statistic: 0.0788, p-value: 0.1000
Série apresenta alta assimetria ou curtose. Aplicando transformação logarítmica.

Testando estacionariedade após transformação (log1p):
ADF Statistic: -3.6462, p-value: 0.0049
KPSS Statistic: 0.0689, p-value: 0.1000
Série já estacionária após transformação.

Resumo da transformação: log1p, número de diferenças aplicadas: 0


## Treinamento e Previsão: ARIMA

Ajuste um modelo ARIMA simples (sem sazonalidade) e gere previsões para o período de teste.

In [15]:
import itertools
import random
import warnings
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")

# Use transformed train series for ARIMA
arima_order = (1, ndiffs, 1)
print(f"Fitting ARIMA with order: {arima_order}")
arima_model = ARIMA(train_transformed, order=arima_order)
arima_fit = arima_model.fit()
arima_forecast_trans = arima_fit.forecast(steps=test_size)
warnings.filterwarnings("default")

# Invert transformations for ARIMA forecast
arima_forecast = invert_transform(arima_forecast_trans, trans_type, lam, shifted, min_val)

arima_metrics = {}
arima_metrics['mae'] = np.mean(np.abs(test_series.values - arima_forecast))
arima_metrics['rmse'] = np.sqrt(np.mean((test_series.values - arima_forecast)**2))
arima_metrics['r2'] = 1 - np.sum((test_series.values - arima_forecast)**2) / np.sum((test_series.values - np.mean(test_series.values))**2)

arima_preds_file = save_predictions(
    y_true=test_series.values,
    y_pred=arima_forecast,
    dates=test_weeks,
    city_name=selected_city_name,
    model_name='arima',
    output_dir=results_dir
)
arima_metrics_file = save_metrics(
    metrics=arima_metrics,
    city_name=selected_city_name,
    model_name='arima',
    output_dir=results_dir,
    params={'order': arima_order}
)

# Plot ARIMA forecast and errors
plot_forecast(
    y_true=test_series.values,
    y_pred=arima_forecast,
    dates=test_weeks,
    title=f"ARIMA Forecast for {selected_city_name}",
    output_path=os.path.join(results_dir, 'arima_forecast_plot.png')
)
plot_forecast_error(
    y_true=test_series.values,
    y_pred=arima_forecast,
    title=f"ARIMA Forecast Errors for {selected_city_name}",
    output_path=os.path.join(results_dir, 'arima_error_plot.png')
)

print(f"\nMétricas ARIMA ajustado automaticamente:")
print(f"MAE: {arima_metrics['mae']:.2f}")
print(f"RMSE: {arima_metrics['rmse']:.2f}")
print(f"R2: {arima_metrics['r2']:.3f}")

Fitting ARIMA with order: (1, 0, 1)


TypeError: plot_forecast() got an unexpected keyword argument 'y_true'

## Treinamento e Previsão: SARIMA com Transformações

O modelo será ajustado considerando as transformações e diferenciações necessárias detectadas na célula anterior. O forecast será revertido para a escala original automaticamente.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

# Fixed SARIMA parameters
order = (1, ndiffs, 1)
seasonal_order = (1, 1, 1, 52)

print(f"Ajustando SARIMA com ordem {order} e sazonalidade {seasonal_order}...")

warnings.filterwarnings("ignore")
model = SARIMAX(train_transformed, order=order, seasonal_order=seasonal_order)
fit = model.fit(disp=False)
forecast_trans = fit.forecast(steps=test_size)
warnings.filterwarnings("default")

# Invert transformations for SARIMA forecast
forecast_final = invert_transform(forecast_trans, trans_type, lam, shifted, min_val)

sarima_metrics = {}
sarima_metrics['mae'] = np.mean(np.abs(test_series.values - forecast_final))
sarima_metrics['rmse'] = np.sqrt(np.mean((test_series.values - forecast_final)**2))
sarima_metrics['r2'] = 1 - np.sum((test_series.values - forecast_final)**2) / np.sum((test_series.values - np.mean(test_series.values))**2)

sarima_preds_file = save_predictions(
    y_true=test_series.values,
    y_pred=forecast_final,
    dates=test_weeks,
    city_name=selected_city_name,
    model_name='sarima',
    output_dir=results_dir
)
sarima_metrics_file = save_metrics(
    metrics=sarima_metrics,
    city_name=selected_city_name,
    model_name='sarima',
    output_dir=results_dir,
    params={'order': order, 'seasonal_order': seasonal_order}
)

# Plot SARIMA forecast and errors
plot_forecast(
    y_true=test_series.values,
    y_pred=forecast_final,
    dates=test_weeks,
    title=f"SARIMA Forecast for {selected_city_name}",
    output_path=os.path.join(results_dir, 'sarima_forecast_plot.png')
)
plot_forecast_error(
    y_true=test_series.values,
    y_pred=forecast_final,
    title=f"SARIMA Forecast Errors for {selected_city_name}",
    output_path=os.path.join(results_dir, 'sarima_error_plot.png')
)

print(f"\nMétricas SARIMA ajustado automaticamente:")
print(f"MAE: {sarima_metrics['mae']:.2f}")
print(f"RMSE: {sarima_metrics['rmse']:.2f}")
print(f"R2: {sarima_metrics['r2']:.3f}")

Ajustando SARIMA com ordem (1, 0, 1) e sazonalidade (1, 1, 1, 52)...


KeyboardInterrupt: 